Notebook by:

* Lorenzo Pannacci 1948926
* Stefano Rinaldi 1945551

## Startup

In [ ]:
######################
# LIBRARIES DOWNLOAD #
######################

install_packages = False
if install_packages:
    %pip install numpy pandas

In [1]:
####################
# LIBRARIES IMPORT #
####################

import numpy as np
import pandas as pd
from collections import defaultdict
from collections import Counter
import random

## 1 - Recommendation sytem 

### 1.1 - top 10 films

The aim is to find the top 10 film with most clicks for each user.

<code style="background:red;color:black"> this is what I am going to do step by step:</code>

1) Load the data <br>
2) Fix the dataset (if necessary) <br>
3) Group by user and movie <br>
4) Count the clicks <br>
5) Sort the data by click count for each user.<br>
6) Extract the top 10 movies for each user and take title and genre.<br>


In [3]:
# 1)/2) load the data and fix the dataset
dataset = pd.read_csv('vodclickstream_uk_movies_03.csv')

#clean it
dataset = dataset.drop("Unnamed: 0", axis=1)

# Convert datetime column to a date
dataset.datetime = pd.to_datetime(dataset.datetime)

# Convert release_date column to a date
dataset.release_date = pd.to_datetime(dataset.release_date, errors="coerce")

# Extract genres column to a new column genre_list that includes a list with all genres
dataset["genres_list"] = dataset.genres.apply(lambda row: [word.strip() for word in row.split(",")])

#retrieve all unique film genres
unique_genres = set()

dataset["genres_list"].apply(lambda row: [unique_genres.add(value) for value in row])

0                           [None, None, None]
1                     [None, None, None, None]
2                                 [None, None]
3                                 [None, None]
4         [None, None, None, None, None, None]
                          ...                 
671731                                  [None]
671732          [None, None, None, None, None]
671733                      [None, None, None]
671734                            [None, None]
671735                            [None, None]
Name: genres_list, Length: 671736, dtype: object

In [4]:
# 3)/4)/5)/6) group by user and movie, count clicks and sort them

#create a new dataframe with the information I need
users = dataset[["genres_list", "user_id","title", "genres"]]

#group user and movie and count the click
user_movie_counts =users.groupby(['user_id', 'title']).size().reset_index(name='click_count')

#sort them
user_movie_counts = user_movie_counts.sort_values(by=['user_id', 'click_count'], ascending=[True, False])

#take the top 10 movies for each user
user_movie_counts = user_movie_counts.groupby('user_id').head(10)

#return a new dataframe with the top 10 movies for each user;
# I give as information the user, the titple and the genre of the film (as well as the click count)
#other pieces of information are not asked

result = pd.merge(user_movie_counts, users[['user_id', 'title', 'genres']], on=['user_id', 'title'], how='left')

result['genres'] = result['genres'].astype(str)

#remove duplicates. I have to do this because the previous code gives the right answer, but
#it creates n rows for the x value of the click count.
#so if a film has 9 clicks, it creates 9 times that rows.
result = result.drop_duplicates()

result

,user_id,title,click_count,genres
0,00004e2862,Hannibal,1,"Crime, Drama, Thriller"
1,000052a0a0,Looper,9,"Action, Drama, Sci-Fi, Thriller"
10,000052a0a0,Frailty,3,"Crime, Drama, Thriller"
13,000052a0a0,Jumanji,3,"Adventure, Comedy, Family, Fantasy"
16,000052a0a0,Resident Evil,2,"Action, Horror, Sci-Fi"
...,...,...,...,...
613980,fffeac83be,To the Bone,1,Drama
613981,fffeac83be,True Story,1,"Crime, Drama, Mystery"
613982,ffff2c5f9e,Forks Over Knives,1,Documentary
613983,ffff2c5f9e,Hot Fuzz,1,"Action, Comedy, Mystery, Thriller"


In [5]:
#here you can retrieve the result for a specific user
### CHANGE THE USER HERE TO THE DESIRED ONE
input_user = "1df8e21154"

specific_user = result[result['user_id'] == input_user]

specific_user

,user_id,title,click_count,genres
70879,1df8e21154,Beastly,5,"Drama, Fantasy, Romance"
70884,1df8e21154,Titanic,4,"Drama, Romance"
70888,1df8e21154,Addicted,3,"Drama, Thriller"
70891,1df8e21154,Match Point,3,"Drama, Romance, Thriller"
70894,1df8e21154,My Last Day without You,3,"Comedy, Drama, Romance"
70897,1df8e21154,The Red Thread,3,"Drama, Romance"
70900,1df8e21154,After Earth,2,"Action, Adventure, Sci-Fi"
70902,1df8e21154,Boomerang,2,"Comedy, Drama, Romance"
70904,1df8e21154,Diary of a Mad Black Woman,2,"Comedy, Drama, Romance"
70906,1df8e21154,Lift Me Up,2,Family


<code style="background:orange;color:black">**Little comment**</code> <br>
here we can see the result for a specific user (and before also for all users together). Result is ordered by click count. In this case, we are taking into consideration all clicks and films and users. Even if a click has a duration of 0, it is taken into consideration as the user showed interest to the film and so is useful for both building the next hashing function and for counting the film with most clicks.

### 1.2 - Minhash Signatures

Here I try to group users together into buckets based on the film genres. <br>
<code style="background:red;color:black"> this is what I am going to do step by step:</code>

0) **group users and genres**:<br>
I create a dictionary in which the keys are the *user_id* and the values the *genres* associated to each user. <br>
1) **create a matrix with genres for each user** <br>
I create a matrix in which I assign 1 if the user has seen that genre, 0 if the user has not
2) **compute signatures for each user** <br>
given the matrix, I first create some random hash functions and then I compute the signature for each user.
3) **bucketing**<br>
Now that I have the signatures, I divide users into buckets

In [6]:
# 0)
### Here I create a 2 columns dataframe. one with the users_id and one with the genres. 
#each row has only one genre. this will be needed for creating a dictionary with users as keys and genres as values

small_dataset = dataset[["genres","user_id"]]

exploded_genres = small_dataset.assign(genres=dataset['genres'].str.split(',')).explode('genres').reset_index(drop=True)

exploded_genres['genres'] = exploded_genres['genres'].str.strip()

# Duplicate the 'user_id' column to match the number of rows in the exploded DataFrame
genres_with_user = pd.concat([exploded_genres['user_id'], exploded_genres['genres']], axis=1)

In [7]:
# 0) create a dictionary where I have as keys the user_id and as values all genres related to it

user_genres_dict = {}

#iterare over dataframe
for index, row in genres_with_user.iterrows():
    user_id = row['user_id']
    genre = row['genres']

    #if user_id already exist, append 
    if user_id in user_genres_dict:
        user_genres_dict[user_id].append(genre)
    #if user_id does not exist, create the key
    else:
        user_genres_dict[user_id] = [genre]


#then I convert into a dataframe (it will be necessary later)
user_genres_df = pd.DataFrame(list(user_genres_dict.items()), columns=['user_id', 'genres'])

In [8]:
#1) create the matrix 

#take unique genres of films
#unique_genres was already created before! I am just recalling it for continuity
unique_genres = sorted(list(unique_genres))

#create my matrix. columns are genres and the index is user_id
my_matrix = pd.DataFrame(0, index=dataset["user_id"].unique(), columns=unique_genres)
#put 1 if the user has that genre, put 0 if there is not
for user_id, genres in user_genres_dict.items():
    for genre in genres:
        if genre in unique_genres:
            my_matrix.loc[user_id, genre] = 1

my_matrix.head()

,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,...,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
1dea19f6fe,0,0,0,0,1,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
544dcbc510,0,1,0,0,1,0,0,1,0,1,...,0,0,1,0,0,0,0,1,0,0
7cbcc791bf,1,1,1,0,1,1,0,0,1,1,...,0,0,1,0,0,0,0,1,0,0
ebf43c36b6,1,1,1,1,1,1,0,1,1,1,...,0,0,0,1,0,0,0,1,0,0
a57c992287,1,1,1,1,1,1,1,1,1,1,...,0,0,0,1,1,1,0,1,1,1


In [9]:
#2) compute signature for all users

#I create my hash function.
#I give a number of my choice and then that number of functions are created. the coefficient x and y are changed every time.
#basically in the next function I have a fixed equation but every time the parameters (x,y) are changed randomly
def generate_hash(n_hashes):
    hash_functions = []

    for k in range(n_hashes):
        x = random.randint(1, 100)
        y = random.randint(0, 100)
        hash_functions.append((x, y))

    return hash_functions

#calculate signature 
def calculate_signature(user_preferences, n_hashes, n_genres, list_functions):
    signature = [float('inf')] * n_hashes

    #for every element of my matrix (which is actually a dataframe)
    #for every genre index and the binary encoding (0, 1)
    for index, present in enumerate(user_preferences):
        #for each genre do the hash functions
        for k, hash_function in enumerate(list_functions):
            x, y = hash_function
            hash_value = (x * index * present + y) % n_genres
            #retain only the minimun
            signature[k] = min(signature[k], hash_value)
    #so here I will have the minimum 
    return signature

#my variables
#the more the n_hashes, the better but it will require more time
genres = list(unique_genres)
n_hashes = 15
n_genres = len(genres)
list_functions = generate_hash(n_hashes)

#dictionary that will store the signatures for each user
signatures = {}

#I calculare the signature for each user in the matrix (which is a dataframe actually)
for user_id, user_preferences in my_matrix.iterrows():
    signature = calculate_signature(user_preferences, n_hashes, n_genres, list_functions)
    signatures[user_id] = signature

In [35]:
#3) divide users into buckets basing on their signature 

def bucketing(signatures, n_hashes, n_bands):
    #number of rows per band
    n_rows = n_hashes // n_bands

    #dictionary that will store the buckets
    buckets = defaultdict(list)

    #for each user:
    for user_id, signature in signatures.items():
        for band_index in range(n_bands):
            #calculates the start and end indices for the current band.
            start = band_index * n_rows
            end = (band_index + 1) * n_rows

            #extract the band from the minhash signature and convert it to a tuple
            band = tuple(signature[start:end])

            #append the user to the corresponding bucket based on the value.
            buckets[band].append(user_id)

    return buckets

n_bands= 6
buckets = bucketing(signatures, n_hashes, n_bands)

### 1.3 - Locality-Sensitive Hashing (LSH)

<code style="background:red;color:black"> this is what I am going to do step by step:</code> <br>

1) **input the user** <br>
input the user to which you wish to suggest 5 films. <br>
2) **Identify the similar users** <br>
Given my user, I want to find all other users that share with my user the same bucket(s) as they must share a common sense of genre film. For the users I find, I count in how many buckets they appear (in the sense that I count how many bucket they share with my user). The more buckets they share, the more probability they are very similar.<br>
3) **select the users** <br>
From the similar users I just found (from the previous step, so the users that share the most buckets with my user) now I compute similarity and thake the top 2 users. They will be used for suggesting films.
5) **suggest common films** <br>
If these two users have any movies in common, recommend those movies in order based on the total number of clicks by these users. <br>
6) **suggest other films** <br>
If there are no common movies or less than 5 common movies, try to propose the most clicked movies by the most similar user first, followed by the other user. <br>

In [48]:
#1)
#INPUT HERE THE USER TO WHICH YOU WANT TO FIND THE MOST SIMILAR USERS (and therefore suggest to them 5 films)
user_id_input = "20c7acabb0"

In [49]:
#2) fund the similar users 

#The idea is that I give a user, and it count occurrences in the same buckets of the other users.
#If my input user and the other users appears into a lot of common buckets, it is very likely that they have same interests 
#The output is a list sorted in descending order of occurrences.

def similar_users(user_id, n_users=10):
    #make a list with all users into the buckets in which the input user_id appears
    user_buckets = [bucket for bucket in buckets.values() if user_id in bucket]
    all_users = [user for bucket in user_buckets for user in bucket if user != user_id]

    #count occurrences of users 
    user_occurrences = Counter(all_users)

    #sort 
    sorted_user_occurrences = user_occurrences.most_common(n_users)
    return sorted_user_occurrences

most_similar_users = similar_users(user_id_input)

In [50]:
#3) select users by similarity 

#given the 10 users I received from the previous filtering, 
#I compute on them jaccard similarity and sort them, so that I am 100% sure that they are ordered into the right way.
#then, I take the 2 users with the highest score

def calculate_jaccard_similarity(user1_genres, user2_genres):
    #jaccard similarity
    intersection = len(set(user1_genres) & set(user2_genres))
    union = len(set(user1_genres).union(user2_genres))
    return intersection / union

input_user_genres = user_genres_df[user_genres_df['user_id'] == user_id_input]['genres'].iloc[0]

#store the similarity result
similarities = []

for similar_user_id, _ in most_similar_users:
    similar_user_genres = user_genres_df[user_genres_df['user_id'] == similar_user_id]['genres'].iloc[0]
    
    jaccard_similarity = calculate_jaccard_similarity(input_user_genres, similar_user_genres)
    
    similarities.append({'user_id': similar_user_id, 'similarity': jaccard_similarity})

similarities_df = pd.DataFrame(similarities)
similarities_df = similarities_df.sort_values("similarity", ascending= False)
print(similarities_df)

#take the top 2 users
users = similarities_df.user_id.head(2)

user_1 =  users.iloc[0]
user_2 =  users.iloc[1]

      user_id  similarity
6  dd77cda733    0.642857
5  9c1ab7b359    0.636364
2  775a8ab995    0.600000
7  7ddbaf4702    0.545455
8  1eb52c1c77    0.500000
4  60455cc93f    0.411765
1  3a142199a8    0.400000
0  25a4de9d06    0.333333
3  f9a35f2935    0.294118
9  68164a256f    0.142857


In [51]:
# 4) If these two users have any movies in common, recommend those movies based on the total number of clicks by these users
#extract common movies

# Filter data for each user
user1_movies = user_movie_counts[user_movie_counts['user_id'] == user_1]
user2_movies = user_movie_counts[user_movie_counts['user_id'] == user_2]

#merge the dataframes such that I have all the movies together
merged_users = pd.concat([user1_movies, user2_movies])

#NOW, I suggest all movies that are in common and I order them by clicks count.

user_groups = merged_users.groupby('user_id')['title'].apply(set)

# Find the common titles between the two users
common_titles = set.intersection(user_groups.iloc[0], user_groups.iloc[1])

common_df = merged_users[merged_users['title'].isin(common_titles)].copy()

#count clicks for each film
common_df['sum_click_count'] = common_df.groupby('title')['click_count'].transform('sum')

#sort by the sum of cliks
sorted_common_df = common_df.sort_values(by='sum_click_count', ascending=False)

#films in common between the two users, drop dublicates
sorted_common_df = sorted_common_df.drop_duplicates(subset='title')
sorted_common_df = sorted_common_df[["title","sum_click_count"]]
sorted_common_df

,title,sum_click_count
432380,The Big Short,2


In [52]:
### here I do the following:
#If the common films are 5, we are done.
#if they are less than 5, I suggest the remaining ones from the user by greatest similarity.
#if the user with greatest similarity has no more other films, I will pass to the second user.
#ex: if the common films are 3, I will suggest only 2 films from the user with greatest similarity / the second user.

#I check if it has more than 5 rows. if it is the case, I suggest the first 5. if it is not the case, I do the following chunks
value = True
key = sorted_common_df.shape[0]


if sorted_common_df.shape[0] >= 5:
    value = True
    print(sorted_common_df['title'].head(5).to_list())
else:
    value = False

In [53]:
# 6) suggests other films
# if there are less than 5 films, then suggest others in base of the user with the most similarity:


#I create a new dataframe with the films of the two users, but without the one(s) in common
final = 5
sub_key = final - key

common_titles = set(merged_users['title'].value_counts()[merged_users['title'].value_counts() > 1].index)

#remove rows with common titles
filtered_df = merged_users[merged_users['title'].isin(common_titles) == False]

new_df = filtered_df.head(sub_key)

#first I have the common films, then I have the only-one-user films
recomended_films = pd.concat([sorted_common_df, new_df])

In [54]:
#since they are ordered, I just take the first 5 for the result
#this is my final result.
print(recomended_films['title'].head(5).to_list())

['The Big Short', 'Eddie the Eagle', 'London Has Fallen', 'Minimalism: A Documentary About the Important Things', 'Love']


<code style="background:orange;color:black">**little comment**</code>:

As we can see, the function works correctly. our input user *20c7acabb0* has seen Action, Drama, Sport, Music, War, Thriller, Sci-Fi and History exactly as *dd77cda733* and *9c1ab7b359* with few differences in few genres that one has seen and the other has not, meaning that the function is working correctly. <br>
Nevertheless, there are some point that needs to be made. In this way, we are suggesting the films that may be liked by a user, but this type of LSH is not taking into considerations repetitions in the sense that if a user sees 11 film, 10 of which are Comedy and 1 Drama, it counts Comedy and Drama as equals as it will insert 1 in the matrix, whereas the user has a clearly interest into the comedy genre. In order to build a better experience for the user, the function should take into account that and maybe LSH is not the right function for performing that. Nevertheless, if we implement a function as I am suggesting, it will give priority to users who whatches a lot of films making it messier for users who whatches just a couple of films.  


## 2. Grouping Users together!

Now, we will deal with clustering algorithms that will provide groups of Netflix users that are similar among them.

To solve this task, you must accomplish the following stages:

### 2.1 Getting your data + feature engineering

1)  Access to the data found in [this dataset](https://www.kaggle.com/datasets/vodclickstream/netflix-audience-behaviour-uk-movies)

2)  Sometimes, the features (variables, fields) are not given in a dataset but can be created from it; this is known as *feature engineering*. For example, the original dataset has several clicks done by the same user, so grouping data by user_id will allow you to create new features **for each** user:

    a)  Favorite genre (i.e., the genre on which the user spent the most time)

    b)  Average click duration

    c)  Time of the day (Morning/Afternoon/Night) when the user spends the most time on the platform (the time spent is tracked through the duration of the clicks)

    d)  Is the user an old movie lover, or is he into more recent stuff (content released after 2010)?

    e)  Average time spent a day by the user (considering only the days he logs in)

So, in the end, you should have for each user_id five features.

3)  Consider at least 10 additional features that can be generated for each user_id (you can use chatGPT or other LLM tools for suggesting features to create). Describe each of them and add them to the previous dataset you made (the one with five features). In the end, you should have for each user at least 15 features (5 recommended + 10 suggested by you).

In [ ]:
# TODO

### 2.2 Choose your features (variables)!

You may notice that you have plenty of features to work with now. So, it would be best to find a way to reduce the dimensionality (reduce the number of variables to work with). You can follow the subsequent directions to achieve it:

1)  *To normalise or not to normalise? That's the question*. Sometimes, it is worth normalizing (scaling) the features. Explain if it is a good idea to perform any normalization method. If you think the normalization should be used, apply it to your data (look at the available normalization functions in the `scikit-learn` library).

2)  Select **one** method for dimensionality reduction and apply it to your data. Some suggestions are Principal Component Analysis, Multiple Correspondence Analysis, Singular Value Decomposition, Factor Analysis for Mixed Data, Two-Steps clustering. Make sure that the method you choose applies to the features you have or modify your data to be able to use it. Explain why you chose that method and the limitations it may have.

In [ ]:
# TODO

### 2.3 Clustering!

1)  Implement the K-means clustering algorithm (**not** ++: random initialization) using MapReduce. We ask you to write the algorithm from scratch following what you learned in class.

2)  Find an optimal number of clusters. Use at least two different methods. If your algorithms provide diverse optimal K's, select one of them and explain why you chose it.

3)  Run the algorithm on the data obtained from the dimensionality reduction.

4)  Implement **K-means++** from scratch and explain the differences with the results you got earlier.

5)  Ask ChatGPT to recommend other clustering algorithms and choose one. Explain your choice, then ask ChatGPT to implement it or use already implemented versions (e.g., the one provided in the scikit-learn library) and run it on your data. Explain the differences (if there are any) in the results. Which one is the best, in your opinion, and why?

In [ ]:
# TODO

### 2.4 Analysing your results!

You are often encouraged to explain the main characteristics that your clusters have. The latter is called the *Characterizing Clusters* step. Thus, follow the next steps to do it:

1)  Select 2-3 variables you think are relevant to identify the cluster of the customer. For example, Time_Day, Average Click Duration, etc.

2)  Most of your selected variables will be numerical (continuous or discrete), then categorize them into four categories.

3)  With the selected variables, perform pivot tables. On the horizontal axis, you will have the clusters, and on the vertical axis, you will have the categories of each variable. Notice that you have to do one pivot table per variable.

4)  Calculate the percentage by column for each pivot table. The sum of each row (cluster) must be 100.

5)  Interpret the results for each pivot table.

6)  Use any known metrics to estimate clustering algorithm performance (how good are the clusters you found?). Comment on the results obtained.

In [ ]:
# TODO

## 3. Bonus Question

We remind you that we consider and grade the bonuses only if you complete the entire assignment.

[Density-based clustering](https://wires.onlinelibrary.wiley.com/doi/epdf/10.1002/widm.30) identifies clusters as regions in the data space with high point density that are separated from other clusters by regions of low point density. The data points in the separating regions of low point density are typically considered noise or outliers. Typical algorithms that fall into this category are [OPTICS](https://dl.acm.org/doi/pdf/10.1145/304181.304187) and [DBSCAN](https://cdn.aaai.org/KDD/1996/KDD96-037.pdf).

1)  Ask ChatGPT (or any other LLM tool) to list three algorithms for Density-Based Clustering. Choose one and use it on the same dataset you used in 2.3. Analyze your results: how different are they from the centroid-based version?

__Note__: You can implement your algorithm from scratch or use the one implemented in the scikit-learn library; the choice is up to you!

**Ask ChatGPT (or any other LLM tool) to list three algorithms for Density-Based Clustering:** <br>
1) DBSCAN (Density-Based Spatial Clustering of Applications with Noise)<br>
2) OPTICS (Ordering Points To Identify the Clustering Structure) <br>
3) HDBSCAN (Hierarchical Density-Based Spatial Clustering of Applications with Noise)<br>

here what I am going to do: <br>
1) load the dataset of 2.3 <br>
2) apply the dataset to the chosen algorithm (OPTICS) <br>
3) compare the results

In [ ]:
#load the dataset

## 4. Command Line Question

Here is another command line question to enjoy. We previously stated that using the command line tools is a skill that Data Scientists must master.

In this question, you should use any command line tool that you know to answer the following questions using the same dataset that you have been using so far:
  + What is the most-watched Netflix title?
  + Report the average time between subsequent clicks on Netflix.com
  + Provide the ID of the user that has spent the most time on Netflix
    
__Important note:__ You may work on this question in any environment (AWS, your PC command line, Jupyter notebook, etc.), but the final script must be placed in CommandLine.sh, which must be executable. Please run the script and include a __screenshot__ of the <ins>output</ins> in the notebook for evaluation.  

In [ ]:
# TODO

## 5. Algorithmic Question 

Federico studies in a demanding university where he has to take a certain number $N$ of exams to graduate,  but he is free to choose in which order he will take these exams. Federico is panicking since this university is not only one of the toughest in the world but also one of the weirdest. His final grade won't depend at all on the mark he gets in these courses: there's a precise evaluation system. 

He was given an initial personal score of $S$ when he enrolled, which changes every time he takes an exam: now comes the crazy part.
He soon discovered that every of the $N$ exams he has to take is assigned a mark $p$. Once he has chosen an exam, his score becomes equal to the mark $p$, and at the same time, the scoring system changes:
+ If he takes an "easy" exam (the score of the exam being less than his score), every other exam's mark is increased by the quantity $S - p$.
+ If he takes a "hard" exam (the score of the exam is greater than his score), every other exam's mark is decreased by the quantity $p - S$.
  
So, for example, consider $S=8$ as the initial personal score. Federico must decide which exam he wants to take, being $[5,7,1]$ the marks list. If he takes the first one, being $5 < 8$ and $8 - 5 = 3$, the remaining list now becomes $[10,4]$, and his score is updated as $S = 5$.

In this chaotic university where the only real exam seems to be choosing the best way to take exams, you are the poor student advisor who is facing a long queue of confused people who need some help. Federico is next in line,  and he comes up in turn with an inescapable question: he wants to know which is the highest score possible he could get. 

a) Fortunately, you have a computer app designed by a brilliant student. Federico wants you to show him the code which this app is based on
because he wants to do paid counseling for other desperate students: in a *recursive* fashion, the helped helps the helpable.

b) Federico is getting angry because he claims that your code is slow! Show him formally with a big-O notation that he is as crazy as this university! 

c) If, unfortunately, Federico is right in the grip of madness, he will threaten you to optimize the code through a different approach. 
You should end this theater of the absurd by any means! (And again, formally prove that you improved time complexity)

d) Ask chatGPT for a third (optimized) implementation and analyze again its time complexity. Be careful (and crafty) in defining the prompt, and challenge the machine in this coding question!

Here are some input/output examples (the first value is the initial personal score, and the second line contains the list of marks): 

__Input 1__
```
8
5 7 1 
```

__Output 1__
```
11
```

__Input 2__
```
25
18 24 21 32 27
```

__Output 2__
```
44
```

__Input 3__
```
30
13 27 41 59 28 33 39 19 52 48 55 79
```

__Output 3__
```
109
```

In [ ]:
# TODO